In [ ]:
!pip install ./darkflow --user

Processing c:\users\elite\pycharmprojects\opencv\tracking-with-darkflow\darkflow
  Running setup.py bdist_wheel for darkflow: started
  Running setup.py bdist_wheel for darkflow: finished with status 'done'
  Stored in directory: C:\Users\Elite\AppData\Local\Temp\pip-ephem-wheel-cache-k8z5fe_u\wheels\94\8e\d5\44bcaa6b10bc5a89a3dd26ffe285e79abbf93d290533c6e8fa
Successfully built darkflow


In [1]:
from darkflow.net.build import TFNet
import cv2
import numpy as np
from sort.sort import *

C:\dev\python\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
options = {"model": "darkflow/cfg/yolo.cfg", "load": "darkflow/bin/yolov2.weights", "threshold": 0.25}

In [3]:
tfnet = TFNet(options)

C:\Users\Elite\AppData\Roaming\Python\Python36\site-packages\darkflow\dark\darknet.py:54: UserWarning: darkflow/cfg/yolov2.cfg not found, use darkflow/cfg/yolo.cfg instead
  cfg_path, FLAGS.model))


Parsing darkflow/cfg/yolo.cfg
Loading darkflow/bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.022905826568603516s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  

In [4]:
def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        if result['label'] != 'person':
            continue
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.25:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255, 0, 0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8,
                                   (0, 230, 0), 1, cv2.LINE_AA)

    return newImage

In [10]:
def getPoint(predictions):
    bboxes = []
    for result in predictions:
        if result['label'] != 'car':
            continue
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']
        
        score = result['confidence']

        bbox = [top_x, top_y, btm_x, btm_y, score]
        bboxes.append(bbox)
        
    bboxes = np.array(bboxes)
        
    return bboxes

In [6]:
def id_box(image, boxes):
    image = np.copy(image)
    
    for box in boxes:
        top_x = int(box[0])
        top_y = int(box[1])
        btm_x = int(box[2])
        btm_y = int(box[3])
        
        track_id = str(int(box[4]))
        
        image = cv2.rectangle(image, (top_x, top_y), (btm_x, btm_y), (255, 255, 255), 1)
        image = cv2.putText(image, track_id, (top_x, top_y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (255, 255, 255), 1)
        
    return image


In [12]:
cap = cv2.VideoCapture('darkflow/car2.mp4')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
tracker = Sort()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret == True:
        (H, W) = frame.shape[:2]
        matrix = cv2.getRotationMatrix2D((W/2, H/2), 270, 1)
        frame = cv2.warpAffine(frame, matrix, None)
        
        frame = np.asarray(frame)
        results = tfnet.return_predict(frame)
        dets = getPoint(results)
        trackers_id = tracker.update(dets)
        tracking = id_box(frame, trackers_id)
        
        
        # Display the resulting frame
        cv2.imshow('frame', tracking)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [48]:
point

[[77, 148, 578, 476], [23, 339, 141, 441], [1, 385, 44, 441]]

In [47]:
trackers_id

array([[  6., 292.,  97., 388.,  44.],
       [258., 281., 341., 363.,  43.],
       [141.,  89., 639., 479.,  42.]])